In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("spam.csv", encoding="latin1")

In [6]:
df.sample(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
1489,ham,Ok i juz receive..,NaN,NaN,NaN
4140,ham,Beautiful truth : Expression of the face could...,NaN,NaN,NaN
915,ham,I could ask carlos if we could get more if any...,NaN,NaN,NaN
5259,ham,Can help u swoop by picking u up from wherever...,NaN,NaN,NaN
4358,ham,ÌÏ only send me the contents page...,NaN,NaN,NaN


In [7]:
df.shape

(5572, 5)

In [8]:
# 1. Data cleaning 
# 2. EDA
# 3. Text preprocessing
# 4. Model Building 
# 5. Evaluation
# 6. Improvement

## 1. Data Cleaning

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [12]:
# drop last 3 columns
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

In [13]:
df.sample(5)

,v1,v2
3028,ham,What is your record for one night? :)
2431,ham,That's good. Lets thank God. Please complete t...
1744,spam,Someone has conacted our dating service and en...
3902,ham,Waiting in e car 4 my mum lor. U leh? Reach ho...
4875,spam,lyricalladie(21/F) is inviting you to be her f...


In [14]:
# renaming the cols
df.rename(columns = {'v1' : 'target','v2':'text'},inplace = True)

In [15]:
df.sample(5)

,target,text
5288,ham,An excellent thought by a misundrstud frnd: I ...
3504,ham,"life alle mone,eppolum oru pole allalo"
5334,ham,"Garbage bags, eggs, jam, bread, hannaford whea..."
5275,ham,Oh yeah clearly it's my fault
2439,ham,Rightio. 11.48 it is then. Well arent we all u...


In [16]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [18]:
df['target'] = encoder.fit_transform(df['target'])

In [19]:
df.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [20]:
# missing values
df.isnull().sum()

target    0
text      0
dtype: int64

In [23]:
#check for duplicate values
df.duplicated().sum()

403

In [25]:
#remove duplicate values
df = df.drop_duplicates(keep="first")

In [26]:
df.duplicated().sum()

0

In [27]:
df.shape

(5169, 2)

## 2. EDA